In [ ]:
from google.colab import drive
# Mount google drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Second Competition/
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Second Competition
Anything_goes.ipynb  test.ipynb    train3.tsv
NLTK_HMM.ipynb       train-02.zip  train.tsv


In [ ]:
import pandas as pd
# Load dataset file
df_total = pd.read_csv("train.tsv", sep='\t', names=["word", "tag"])
df_total.head()

,word,tag
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N


In [ ]:
df2 = df_total[:1000]
df2.describe()
print((df2['tag'].value_counts()))
lbl = df2['tag'].value_counts().max()
max_label = df2['tag'][lbl]
max_label

N    861
S    105
U     22
H      8
T      4
Name: tag, dtype: int64


'N'

In [ ]:
df2[df2.tag == "T"]

,word,tag
196,ultach,T
239,aon,T
265,aontú,T
405,ábhar,T


In [ ]:
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('universal_tagset')
brownwords = nltk.corpus.brown.tagged_words(categories='news', tagset='universal')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [ ]:
split = int(len(df3)*0.9)
train = df3[:split]
test = df3[split:]
len(train)

900

In [ ]:
unigram_tagger = nltk.UnigramTagger(train)
unigram_tagger.accuracy(test)

0.54

In [ ]:
# Loading libraries

from nltk.tag import SequentialBackoffTagger
from nltk.tag import DefaultTagger 
from nltk.tag import UnigramTagger 

back_tagger = DefaultTagger(max_label)
Uni_tagger = UnigramTagger(train, backoff = back_tagger)
Uni_tagger.evaluate(test)

Baseline RNN

In [ ]:
vocab = list(df2['word'])
vocab.append('<PAD>')

tags = list(df2['tag'])
tags.append('<PAD>')
 
tags

In [ ]:
train

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 60
word2index = {w: i for i, w in enumerate(vocab)}
onehot = [[word2index[w[0]] for w in s] for s in df3]

In [ ]:
from tensorflow.keras.utils import to_categorical
onehot_y = [[tag2index[w[1]] for w in s] for s in df3]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
y = to_categorical(y, num_classes=len(tags))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train

array([[ 567, 1000, 1000, ..., 1000, 1000, 1000],
       [ 762, 1000, 1000, ..., 1000, 1000, 1000],
       [ 640, 1000, 1000, ..., 1000, 1000, 1000],
       ...,
       [ 975, 1000, 1000, ..., 1000, 1000, 1000],
       [ 986, 1000, 1000, ..., 1000, 1000, 1000],
       [ 994, 1000, 1000, ..., 1000, 1000, 1000]], dtype=int32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
model = Sequential()
model.add(TimeDistributed(Dense(len(tags), activation="softmax")))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 50)            50050     
                                                                 
 bidirectional (Bidirectiona  (None, 60, 200)          120800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 60, 1001)         201201    
 ibuted)                                                         
                                                                 
Total params: 372,051
Trainable params: 372,051
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
n_epochs= 1
# run fit on Colab or hopper
history = model.fit(X_train, y_train, batch_size=16, epochs=n_epochs, validation_split=0.1, verbose=1)

51/51 [==============================] - 16s 315ms/step - loss: 0.0395 - accuracy: 0.9835 - val_loss: 0.0384 - val_accuracy: 0.9870


In [ ]:
import numpy
test_sent=41
pred = model.predict(numpy.array([X_test[test_sent]]))
for i,ix in enumerate(X_test[test_sent]):
    if ix==len(vocab)-1:
        break
    print("{:20} — {}".format(vocab[ix], tags[p[0][i]]))

a                    — <PAD>
